# Random Decision Trees Regression Example

## Boston housing prices
The objective is to predict the median price of a home in Boston.  The variables are crime rate, zoning information,
proportion of non-retail business, etc.  This dataset has median prices in Boston for 1972.  Even though the data is pretty old, the methodology for analytics is valid for more recent datasets.

<b>The purpose of this demonstration is to show the use of SAP HANA's Predictive Analytics Library to created Random Decision Trees model.</b>

The dataset is from Kaggle. https://www.kaggle.com/c/boston-housing. For tutorials use only.

## Housing Values in Suburbs of Boston in 1972

The <font color='red'>medv</font> variable is the target variable.
### Data description
The Boston data frame has 506 rows and 14 columns.
This data frame contains the following columns:
1. __crim__: per capita crime rate by town.
2. __zn__: proportion of residential land zoned for lots over 25,000 sq.ft.
3. __indus__: proportion of non-retail business acres per town.
4. __chas__: Charles River dummy variable (1 if tract bounds river; 0 otherwise).
5. __nox__: nitrogen oxides concentration (parts per 10 million).
6. __rm__: average number of rooms per dwelling.
7. __age__: proportion of owner-occupied units built prior to 1940.
8. __dis__: weighted mean of distances to five Boston employment centres.
9. __rad__: index of accessibility to radial highways.
10. __tax__: full-value property-tax rate per \$10000
11. __ptratio__: pupil-teacher ratio by town.
12. __black__: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.
13. __lstat__: lower status of the population (percent).
14. __medv__: median value of owner-occupied homes in $1000s.
</td></tr></table>

### Factoids
The prices in Boston across years is below.  If we had a historical dataset, an analysis could be done to account for the macro trends as well.

The second graph shows the intuition we have with respect to prices in relation to crime rate.  It is expected that house prices will be lower in areas where crime rates are higher.

The third figure is a chart showing how inflation may affect prices.  So, for deeper analysis and prediction, we may want to consider inflation.

In this notebook, these factors are not considered.  They are here to demonstrate the need for deep domain analysis.

<table><tr>
<td><img src="images/boston_prices_by_year.png" alt="Boston home prices" title="Boston housing prices" style="float:left;" /></td>
<td><img src="images/Crime-Rate-and-Median-House-Prices.png" alt="Boston home prices" title="Boston housing prices"  /></td>
<td><img src="images/Inflation_Adjusted_Housing_Prices_1890_2006.jpg" alt="Inflation adjusted prices" title="Inflation adjusted prices" style="float:left;" />
</td></tr></table>


In this notebook, we will use the dataset for Boston housing prices and predict the price based on numerous factors.

In [9]:
from hana_ml import dataframe
from hana_ml.algorithms.pal import clustering
from hana_ml.algorithms.pal import trees
import numpy as np
import matplotlib.pyplot as plt
import logging

## Load data
The data is loaded into 4 tables, for full, training, validation, and test sets:
<li>BOSTON_HOUSING_PRICES</li>
<li>BOSTON_HOUSING_PRICES_TRAINING</li>
<li>BOSTON_HOUSING_PRICES_VALIDATION</li>
<li>BOSTON_HOUSING_PRICES_TEST</li>

To do that, a connection is created and passed to the loader.

There is a config file, config/e2edata.ini that controls the connection parameters and whether or not to reload the data from scratch.  In case the data is already loaded, there would be no need to load the data.  A sample section is below.  If the config parameter, reload_data is true then the tables for test, training, and validation are (re-)created and data inserted into them.

Although this ini file has other sections, please do not modify them. Only the [hana] section should be modified.

#########################<br>
[hana]<br>
url=host.sjc.sap.corp<br>
user=username<br>
passwd=userpassword<br>
port=3xx15<br>
#########################<br>

In [10]:
from data_load_utils import DataSets, Settings
url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
connection_context = dataframe.ConnectionContext(url, port, user, pwd)
full_tbl, training_tbl, validation_tbl, test_tbl = DataSets.load_boston_housing_data(connection_context)

Table BOSTON_HOUSING_PRICES exists and data exists


# Create Data Frames
Create the data frames for the full, test, training, and validation sets.

Let us also do some dtaa exploration.

## Define Datasets - Training, validation, and test sets
Data frames are used keep references to data so computation on large data sets in HANA can happen in HANA.  Trying to bring the entire data set into the client will likely result in out of memory exceptions.

The original/full dataset is split into training, test and validation sets.  In the example below, they reside in different tables.

In [11]:
full_set = connection_context.table(full_tbl)
training_set = connection_context.table(training_tbl)
validation_set = connection_context.table(validation_tbl)
test_set = connection_context.table(test_tbl)

## Simple Exploration
Let us look at the number of rows in the data set

In [12]:
print('Number of rows in full set: {}'.format(full_set.count()))
print('Number of rows in training set: {}'.format(training_set.count()))
print('Number of rows in validation set: {}'.format(validation_set.count()))
print('Number of rows in test set: {}'.format(test_set.count()))

Number of rows in full set: 506
Number of rows in training set: 253
Number of rows in validation set: 202
Number of rows in test set: 51


### Let's look at the columns

In [13]:
print(full_set.columns)

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'BLACK', 'LSTAT', 'MEDV', 'ID']


### Let's look at the data types

In [14]:
full_set.dtypes()

[('CRIM', 'DECIMAL', 12, 12, 12, 5),
 ('ZN', 'DECIMAL', 7, 7, 7, 3),
 ('INDUS', 'DECIMAL', 7, 7, 7, 2),
 ('CHAS', 'SMALLINT', 5, 5, 5, 0),
 ('NOX', 'DECIMAL', 10, 10, 10, 4),
 ('RM', 'DECIMAL', 8, 8, 8, 3),
 ('AGE', 'DECIMAL', 7, 7, 7, 3),
 ('DIS', 'DECIMAL', 11, 11, 11, 4),
 ('RAD', 'TINYINT', 3, 3, 3, 0),
 ('TAX', 'SMALLINT', 5, 5, 5, 0),
 ('PTRATIO', 'DECIMAL', 6, 6, 6, 2),
 ('BLACK', 'DECIMAL', 9, 9, 9, 3),
 ('LSTAT', 'DECIMAL', 7, 7, 7, 2),
 ('MEDV', 'DECIMAL', 6, 6, 6, 2),
 ('ID', 'INT', 10, 10, 10, 0)]

### Set up the features and labels for the model

In [15]:
features=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'BLACK', 'LSTAT']
label='MEDV'

# Create model using training data
For demonstration, we will create two models, model and model_with_id, one where we have a unique id in the training set and one where there is none.

We are using Random Decision Trees regression and SVM routines in this example

Documentation is <a href="https://help.sap.com/http.svc/rc/DRAFT/3f0dbe754b194c42a6bf3405697b711f/2.0.031/en-US/html/index.html">here</a>

## Preprocessing
SAP HANA Predictive Analytics Library takes DOUBLE and INTEGER data types for most numeric types.  Since we have DECIMALs and TINYINTs in our data set, we cast them to the types required by PAL.

In [16]:
# Cast to correct types so PAL can consume it.
dfts = training_set.cast(['CRIM', "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "PTRATIO", "BLACK", "LSTAT", "MEDV"], "DOUBLE")
dfts = dfts.cast(["CHAS", "RAD", "TAX"], "INTEGER")
dfts = dfts.to_head("ID")
dfts.head(5).collect()

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,196,0.01381,80.0,0.46,0,0.422,7.875,32.0,5.6484,4,255,14.4,394.23,2.97,50.0
1,124,0.15038,0.0,25.65,0,0.581,5.856,97.0,1.9444,2,188,19.1,370.31,25.41,17.3
2,60,0.10328,25.0,5.13,0,0.453,5.927,47.2,6.9320,8,284,19.7,396.90,9.22,19.6
3,76,0.09512,0.0,12.83,0,0.437,6.286,45.0,4.5026,5,398,18.7,383.23,8.94,21.4
4,482,5.70818,0.0,18.10,0,0.532,6.750,74.9,3.3317,24,666,20.2,393.07,7.74,23.7


## Create the model
Although we had seen graphically that only a few features had an impact on housing prices, let us use all the features to create a model.  We will then use the model to check for importance of the features.

In [17]:
# We build the model without IDs.  Project only the features and the label.
df = dfts.select(features, label)
model = trees.RDTRegressor()

In [18]:
model.fit(df, features=features, label=label)

### SQL statements executed
Calling PAL directly would require a number of SQL statements and all that is encapsulated in the Python library functions.

## Model analysis
Let's just see what features are most important.
Note that we are using a sort function.  The property __feature_importances___ is automatically set when the fit() method is called above.

In [19]:
model.feature_importances_.sort(['IMPORTANCE'], desc=True).collect()

,VARIABLE_NAME,IMPORTANCE
0,RM,0.361404
1,LSTAT,0.294479
2,NOX,0.081225
3,CRIM,0.062683
4,PTRATIO,0.044075
5,DIS,0.039123
6,INDUS,0.034039
7,TAX,0.030668
8,AGE,0.021611
9,CHAS,0.013019


__As you can see above, LSTAT, RM, NOX, and PTRATIO seem to have the most impact on prices.__

# Predict using test set
Let us now do some predictions and see how well the model generalizes.

The predict() method always takes a unique identifier to identify the prediction on a specific data row.  This way, the caller (python programmer) can then join with the original data set to get the rest of the values for that unique row.  The test_set has columns of types that PAL does not deal with and therefore the columns are cast to the types that are accepted.

In order to look at the predicted value as well as the true value, the name of the unique identifier for rows in the result table is renamed to PREDICTED_ID.  This result table is joined with the test set so the predicted and true value can be compared.

For the predictions we look at the standard error.  The standard error is defined as the number of standard deviations away the prediction is from the true value.

In [20]:
df_test = test_set.cast(['CRIM', "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "PTRATIO", "BLACK", "LSTAT", "MEDV"], "DOUBLE")
df_test = df_test.cast(["CHAS", "RAD", "TAX"], "INTEGER")
df_test = df_test.to_head("ID")

In [21]:
# Note that we are renaming the column ID in the result of predict()
model.enable_parallel_by_parameter_partitions()
result_df = model.predict(df_test, key= 'ID', features=features, verbose=True).rename_columns({'ID': 'PREDICTED_ID'})
# Note the use of join() method to join two tables.
jdf = result_df.join(test_set, '{}."PREDICTED_ID"={}."ID"'.format(result_df.name, test_set.name), how='inner')

In [22]:
result_df.collect()

,PREDICTED_ID,SCORE,CONFIDENCE
0,225,37.7075,6.618201
1,450,15.3064,2.502259
2,309,28.2401,8.262463
3,13,21.5802,2.100634
4,231,22.6658,3.561870
5,498,21.0172,1.595530
6,141,17.7193,3.181536
7,311,21.1173,1.881078
8,472,20.4302,1.771659
9,496,20.2649,2.806375


### Predictions
Let us look at the predictions.  The predicted values are in 'SCORE' and the actual values are in 'MEDV'.  So, we just rename the 'SCORE' column to 'PREDICTED'

In addition, the column 'CONFIDENCE' is the standard error which is the number of standard deviations away the actual values is from the predicted value.  This column is renamed to 'STANDARD_ERROR'

In [23]:
jdf.select(['ID', 'SCORE', 'MEDV', 'CONFIDENCE']).rename_columns({"CONFIDENCE": "STANDARD_ERROR", "SCORE": "PREDICTED"}).sort("STANDARD_ERROR", desc=False).head(5).collect()

,ID,PREDICTED,MEDV,STANDARD_ERROR
0,347,21.4976,17.2,1.327258
1,346,21.6302,17.5,1.365111
2,85,22.5113,23.9,1.385688
3,498,21.0172,18.3,1.595530
4,484,20.7396,21.8,1.733669


### Out of bag error
Let us look at the out of bag errors which is a method of measuring the prediction error.

Here we look at the first 4 rows

In [24]:
model.oob_error_.head(4).collect()

,TREE_INDEX,ERROR
0,0,6.478819
1,1,6.478819
2,2,6.093921
3,3,6.060785


## Scoring
We now score the results from are test data.  The scoring function we use is R^2.

__In the function below, PAL is not invoked but a query is directly executed against data in HANA__

In [25]:
r2_score = model.score(df_test, key='ID', features=features, label=label)
print("r2 score is {}".format(r2_score))

r2 score is 0.8183424671428448


## Model
The model is available and can be saved for later predictions

In [26]:
#  The generated model is in the database.
model.model_.head(4).collect()

,ROW_INDEX,TREE_INDEX,MODEL_CONTENT
0,0,-1,"<PMML version=""4.0"" xmlns=""http://www.dmg.org/..."
1,1,0,"<PMML version=""4.0"" xmlns=""http://www.dmg.org/..."
2,2,1,"<PMML version=""4.0"" xmlns=""http://www.dmg.org/..."
3,3,2,"<PMML version=""4.0"" xmlns=""http://www.dmg.org/..."


## Close the Connection

In [27]:
connection_context.close()